# DAX: Ключевые функции Power BI
**CALCULATE + контекст = сила DAX**

## Содержание
1. Фильтры и контекст  
2. Агрегация и расчёты  
3. Временные расчёты (YoY)  
4. Условные расчёты  
5. Self-Join и сложные связи  
6. Расширенные функции  
7. Мера vs Вычисляемый столбец  
8. Иерархия дат — лучшие практики  
9. Переменные (VAR) — для читаемости  
10. Обработка ошибок  
11. Динамические заголовки  
12. Фильтрация по мерам  
13. Наиболее полезные функции  
14. Производительность  
15. Шаблон сложной меры  
---

## 1. Фильтры и контекст

```dax
Продажи Москва =
CALCULATE(
    SUM(Sales[Amount]),
    Customers[City] = "Москва"
)
```

> **CALCULATE** — меняет контекст фильтрации  
> **ALL()** — снимает фильтр: `CALCULATE(SUM(...), ALL(Date))`  
> **FILTER()** — сложные условия: `CALCULATE(SUM(...), FILTER(Table, Condition))`

## 2. Агрегация и расчёты

```dax
% от общего =
DIVIDE(
    [Продажи],
    CALCULATE([Продажи], ALL(Sales))
)
```

> **DIVIDE** — безопасное деление  
> **ALL(Sales)** — игнорирует фильтры по строкам Sales

## 3. Временные расчёты (YoY)

```dax
-- Базовая мера
Продажи = SUM(Sales[Amount])

Продажи прошлый год =
CALCULATE(
    [Продажи],
    SAMEPERIODLASTYEAR('Calendar'[Date])
)

YoY % =
DIVIDE(
    [Продажи] - [Продажи прошлый год],
    [Продажи прошлый год]
)
```

> **SAMEPERIODLASTYEAR**, **DATESYTD**, **TOTALYTD** — встроенная временная аналитика  
> **ALL()** — не ставим, если нужна интерактивность (фильтры по регионам, товарам)

## 4. Условные расчёты

```dax
Категория =
SWITCH(
    TRUE(),
    [Продажи] > 10000, "High",
    [Продажи] > 1000, "Medium",
    "Low"
)
```

> **SWITCH(TRUE(), ...)** — аналог CASE WHEN в SQL

## 5. Self-Join и сложные связи

```dax
Имя руководителя =
LOOKUPVALUE(
    Employees[Name],
    Employees[ID], Employees[ManagerID]
)
```

> **LOOKUPVALUE** — как self-join в SQL

## 6. Расширенные функции

- **Агрегация по связанным таблицам**:
  ```dax
  TotalByCategory = SUMX(RELATEDTABLE(Category), Category[Value])
  ```

- **Ранжирование**:
  ```dax
  Rank = RANKX(ALL(Table), [Продажи], , DESC)
  ```

- **Дата-функции**:
  ```dax
  SalesYTD = TOTALYTD([Продажи], 'Calendar'[Date])
  ```

- **IF для условий**:
  ```dax
  Status = IF([Продажи] > 1000, "High", "Low")
  ```

## 7. Мера vs Вычисляемый столбец

| Мера | Вычисляемый столбец |
|------|---------------------|
| Динамична (реагирует на фильтры) | Статична (считается при загрузке) |
| Не занимает память | Занимает место в модели |
| Используй: YoY, % от общего, фильтры | Используй: категории, флаги, ID |

> **Правило:** Если зависит от **контекста** — **мера**. Если **фиксированное значение** — **столбец**.

## 8. Иерархия дат — лучшие практики

```dax
-- Рекомендуемый способ: CALENDARAUTO()
Calendar =
VAR Base = CALENDARAUTO()
RETURN
ADDCOLUMNS(
    Base,
    "Year", YEAR([Date]),
    "Quarter", "Q" & FORMAT([Date], "q"),
    "MonthName", FORMAT([Date], "MMMM", "ru-RU"),
    "MonthNum", MONTH([Date]),
    "MonthYear", FORMAT([Date], "MMM YYYY", "ru-RU"),
    "Week", "Неделя " & WEEKNUM([Date]),
    "Weekday", FORMAT([Date], "dddd", "ru-RU"),
    "Day", DAY([Date])
)
```

**Настройка в Power BI:**
1. Связь: `Calendar[Date]` → `Sales[OrderDate]` (1:many)  
2. **Modeling → Mark as Date Table** → `Date`  
3. Иерархия: `Year → Quarter → MonthName → Day`  
4. `MonthName` → **Sort by Column** → `MonthNum`  
5. **File → Options → Data Load → Disable Auto Date/Time**

> **ISINSCOPE** — скрывать YoY на уровне дня:
> ```dax
> YoY % (умный) = IF(ISINSCOPE('Calendar'[Day]), BLANK(), [YoY %])
> ```

## 9. Переменные (VAR) — для читаемости

```dax
Продажи с дисконтом =
VAR TotalSales = SUM(Sales[Amount])
VAR Discount = 0.1
VAR DiscountedSales = TotalSales * (1 - Discount)
RETURN
IF(DiscountedSales > 1000, DiscountedSales, TotalSales)
```

> **VAR** — однократное вычисление, отладка, читаемость

## 10. Обработка ошибок

```dax
Безопасное деление =
DIVIDE([Продажи], [Количество], 0)

Проверка на пустоту =
IF(ISBLANK([Мера]), "Нет данных", [Мера])
```

> **DIVIDE** — всегда вместо `/`

## 11. Динамические заголовки

```dax
Динамический заголовок =
"Продажи: " & FORMAT([Продажи], "#,##0") & " (" & FORMAT([YoY %], "0%") & " vs пр.год)"
```

## 12. Фильтрация по мерам

```dax
Продажи Топ-5 =
CALCULATE(
    [Продажи],
    TOPN(5, ALL(Products), [Продажи])
)

Продажи без нулей =
CALCULATE([Продажи], Sales[Amount] > 0)
```

## 13. Наиболее полезные функции

**Анализ:** `RANKX`, `TOPN`, `PERCENTILEX.INC`, `AVERAGEX`  
**Даты:** `DATESBETWEEN`, `DATESMTD/QTD/YTD`, `PREVIOUSMONTH`  
**Текст:** `CONCATENATEX`, `SEARCH`, `LEFT/RIGHT/MID`

## 14. Производительность

```dax
-- МЕДЛЕННО:
SUMX(Sales, Sales[Quantity] * Sales[Price])

-- БЫСТРО:
SUM(Sales[TotalAmount]) -- посчитай в Power Query
```

> Избегай `SUMX` на больших таблицах

## 15. Шаблон сложной меры

```dax
Продажи по категориям =
VAR CurrentCategory = SELECTEDVALUE(Products[Category])
VAR TotalSales = [Продажи]
VAR CategorySales =
    CALCULATE([Продажи], ALL(Products), Products[Category] = CurrentCategory)
VAR Percentage = DIVIDE(CategorySales, TotalSales)
RETURN
SWITCH(
    TRUE(),
    ISBLANK(CurrentCategory), TotalSales,
    Percentage < 0.1, "Меньше 10%",
    "Основная категория"
)
```

---
**Источники:** DAX Guide, Power BI Community.  
*Добавляйте свои меры через PR.*